In [1]:
import time as t
import datetime as dt
import pandas as pd
import requests
import numpy as np
import os
import functools
import traceback
import os
import sqlite3
from dotenv import load_dotenv
load_dotenv('../.env')
apikey = os.getenv('bi_api_key')


def retry(max_attempts=10, retry_delay=1):
    def decorator_retry(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            attempt = 0
            while attempt < max_attempts:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Attempt {attempt} failed with error: {e}.\nTraceback: {traceback.format_exc()}")
                    attempt += 1
                    if attempt < max_attempts:
                        t.sleep(retry_delay)
                    else:
                        print("Max retries reached!")
            return None
        return wrapper
    return decorator_retry


@retry(max_attempts=10, retry_delay=1)
def get_price_data(apikey, token, time_from, time_to):
    url = "https://public-api.birdeye.so/defi/ohlcv"
    headers = {"x-chain": "solana", "X-API-KEY": apikey}
    params = {
        "address": token,
        "type": '1m',
        "time_from": time_from,
        "time_to": time_to
    }
    response = requests.get(url, headers=headers, params=params)
    prices_df = pd.DataFrame(response.json()['data']['items'])
    prices_df['address'] = token
    return prices_df

In [ ]:
conn = sqlite3.connect('../dbs/calls.db')

In [8]:
query = "SELECT * FROM calls WHERE launched is NULL" 
df = pd.read_sql_query(query, conn)
df

,id,date,address,expected_launch_time,expected_launch_time_ts,s_mm2,s_ma2,s_fa2,s_q,s_sni,mcap_num,liq_num,launched,launch_time,buy,buy_time,buy_price
0,215618,2024-04-18 14:35:57,CfQSSRk6Md2cnZCzkJu2QjAVua5FzGMCua7CrhQGRURK,In 1 minute,2024-04-18 14:36:57,0,0,0,2,0,9050,18110,None,None,None,None,None
1,215619,2024-04-18 14:37:11,GKMRt3xZznGK4wxoBw3SZfMTwVR9LLFTJxbqMCUvMabg,11 seconds ago,2024-04-18 14:37:00,1,0,0,0,2,1390,2500,None,None,None,None,None
2,215620,2024-04-18 14:37:58,66Yp2kKFLsEjszAMVttwHFDZRZRyGt9kbKoXnuv99Dx3,36 seconds ago,2024-04-18 14:37:22,1,0,0,0,4,1730,1390,None,None,None,None,None
3,215621,2024-04-18 14:38:20,Ecz4mak5jScXrwZkMGJB3mqqpeuJPWPTddJW6Rgmt9Za,27 seconds ago,2024-04-18 14:37:53,0,0,0,0,2,1110,1110,None,None,None,None,None
4,215622,2024-04-18 14:39:15,BdSSmAmWAE8F2yhjAPW91Boa4fQfu98EKt3xxgLKQVxn,36 seconds ago,2024-04-18 14:38:39,1,1,0,0,2,40230,80460,None,None,None,None,None


In [ ]:
for i,row in df.iterrows():
    print('trying: ',i, row['address'])
    tmp_df = get_price_data(apikey, row['address'], int(row['expected_launch_time_ts'] - (60*60*1.5)), int(row['expected_launch_time_ts'] + (60*60*25.5)))
    if not tmp_df.empty:
        price_data = pd.concat([price_data, tmp_df.drop(columns='type')])